#Mount Google Drive

In [ ]:
import sys
import os

from google.colab import drive
drive.mount('/content/gdrive')

# Change working directory to be current folder
import os
os.chdir('/content/gdrive/My Drive/iss/babydetect/')

## Environment setup

In [ ]:
!pip install tensorflow.io
!pip install ffmpeg moviepy
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio

# Sound classification with YAMNet

YAMNet is a deep net that predicts 521 audio event [classes](https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/yamnet_class_map.csv) from the [AudioSet-YouTube corpus](http://g.co/audioset) it was trained on. It employs the
[Mobilenet_v1](https://arxiv.org/pdf/1704.04861.pdf) depthwise-separable
convolution architecture.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_io as tfio

import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython import display

import moviepy.editor as mp

from scipy.io import wavfile
from scipy.signal import resample

Load the Model from TensorFlow Hub.

Note: to read the documentation just follow the model's [url](https://tfhub.dev/google/yamnet/1)

In [ ]:
# Load the model.
yamnet_model = hub.load('YAMNet')

The labels file will be loaded from the models assets and is present at `model.class_map_path()`.
You will load it on the `class_names` variable.

In [ ]:
# solution: loading label names
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

for name in class_names[:5]:
  print(name)

Add a method to convert a loaded audio is on the proper sample_rate (16K), otherwise it would affect the model's results.

Returned wav_data has been normalized to values in [-1.0, 1.0] (as stated in the model's documentation).

In [50]:
@tf.function
def load_wav_16k_mono(filename):
    """ read in a waveform file and convert to 16 kHz mono """
    file_contents = tf.io.read_file(filename)
    
    wav, sample_rate = tf.audio.decode_wav(file_contents,
                                          desired_channels=1)
    
    wav = tf.squeeze(wav, axis=-1)
    
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

## Preparing the sound file

The audio file should be a mono wav file at 16kHz sample rate.

In [ ]:
wav_file_name = './datasets/ESC-50-master/audio/1-187207-A-20.wav'

wav_data = load_wav_16k_mono(wav_file_name)

# Play the audio file.
display.Audio(wav_data, rate=16000)

In [ ]:
plt.plot(wav_data)

## Executing the Model

Now the easy part: using the data already prepared, you just call the model and get the: scores, embedding and the spectrogram.

The score is the main result you will use.
The spectrogram you will use to do some visualizations later.

In [ ]:
# Run the model, check the output.
scores, embeddings, spectrogram = yamnet_model(wav_data)

In [ ]:
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.argmax(class_scores)
infered_class = class_names[top_class]

print(f'The main sound is: {infered_class}')
print(f'The embeddings shape: {embeddings.shape}')

## Visualization

YAMNet also returns some additional information that we can use for visualization.
Let's take a look on the Waveform, spectrogram and the top classes inferred.

In [ ]:
plt.figure(figsize=(10, 6))

# Plot the waveform.
plt.subplot(3, 1, 1)
plt.plot(wav_data)
plt.xlim([0, len(wav_data)])

# Plot the log-mel spectrogram (returned by the model).
plt.subplot(3, 1, 2)
plt.imshow(spectrogram_np.T, aspect='auto', interpolation='nearest', origin='lower')

# Plot and label the model output scores for the top-scoring classes.
mean_scores = np.mean(scores, axis=0)
top_n = 10
top_class_indices = np.argsort(mean_scores)[::-1][:top_n]
plt.subplot(3, 1, 3)
plt.imshow(scores_np[:, top_class_indices].T, aspect='auto', interpolation='nearest', cmap='gray_r')

# patch_padding = (PATCH_WINDOW_SECONDS / 2) / PATCH_HOP_SECONDS
# values from the model documentation
patch_padding = (0.025 / 2) / 0.01
plt.xlim([-patch_padding-0.5, scores.shape[0] + patch_padding-0.5])
# Label the top_N classes.
yticks = range(0, top_n, 1)
plt.yticks(yticks, [class_names[top_class_indices[x]] for x in yticks])
_ = plt.ylim(-0.5 + np.array([top_n, 0]))

## ESC-50 dataset

The ESC-50 dataset, well described here, is a labeled collection of 2000 environmental audio recordings (each 5 seconds long). The data consists of 50 classes, with 40 examples per class

In [ ]:
_ = tf.keras.utils.get_file('esc-50.zip',
                        'https://github.com/karoldvl/ESC-50/archive/master.zip',
                        cache_dir='./',
                        cache_subdir='datasets',
                        extract=True)

## Explore the data

In [ ]:
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
base_data_path = './datasets/ESC-50-master/audio/'

pd_data = pd.read_csv(esc50_csv)
pd_data.head()

## Filter the data

In [8]:
my_classes = ['crying_baby', 'laughing']
saved_model_path = './baby_crying_yamnet'

In [ ]:
map_class_to_id = {'crying_baby':0, 'laughing':1}

filtered_pd = pd_data[pd_data.category.isin(my_classes)]

class_id = filtered_pd['category'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(target=class_id)

full_path = filtered_pd['filename'].apply(lambda row: os.path.join(base_data_path, row))
filtered_pd = filtered_pd.assign(filename=full_path)

filtered_pd.head(10)

## Load the audio files and retrieve embeddings

In [ ]:
filenames = filtered_pd['filename']
targets = filtered_pd['target']
folds = filtered_pd['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

In [ ]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

#main_ds = main_ds.map(lambda a,b,c: tf.py_function(load_wav_for_map, [a, b, c], [tf.float32,tf.int64,tf.int64]))
main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

In [ ]:
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]

  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
#main_ds.element_spec

In [ ]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 4)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 4)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 5)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
print(train_ds)

## Create new model

In [ ]:
new_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), 
                          dtype=tf.float32,
                          name='input_embedding'),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='new_model')

new_model.summary()

In [ ]:
new_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [ ]:
history = new_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Lets run the evaluate method on the test data just to be sure there's no overfitting.

In [ ]:
loss, accuracy = new_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## Test your model

In [ ]:
test_laughing_data = load_wav_16k_mono('./datasets/ESC-50-master/audio/4-155670-A-26.wav')

scores, embeddings, spectrogram = yamnet_model(test_laughing_data)
result = new_model(embeddings).numpy()
print(result)

infered_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {infered_class}')

## Save a model that can directly take a wav file as input

In [ ]:
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

In [ ]:
input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer('YAMNet',
                                            trainable=False, 
                                            name='yamnet')

_, embeddings_output, _ = embedding_extraction_layer(input_segment)

serving_outputs = new_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)

serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)

In [ ]:
tf.keras.utils.plot_model(serving_model)

## Test new model

In [67]:
#test_laughing_data = load_wav_16k_mono('./datasets/ESC-50-master/audio/4-155670-A-26.wav')
#test_crying_data = load_wav_16k_mono('./datasets/ESC-50-master/audio/4-167077-A-20.wav')
aaa = load_wav_16k_mono('./datasets/Babies_Crying.wav')
print(aaa.dtype, aaa.shape)

<dtype: 'float32'> (4084480,)


In [10]:
# loading new model
reloaded_model = tf.saved_model.load(saved_model_path)

In [68]:
# test in new data file
reloaded_results = reloaded_model(aaa)
print(reloaded_results)

baby_sound = my_classes[tf.argmax(reloaded_results)]
print(f'The main sound is: {baby_sound}')

tf.Tensor([ 0.9237363  -0.82834846], shape=(2,), dtype=float32)
The main sound is: crying_baby


## Loading video

In [133]:
#my_clip = mp.VideoFileClip(r"./datasets/Babies_Crying.mp4")

#my_clip.audio.write_audiofile(r"./datasets/Babies_Crying.wav")

## Read audio file

In [115]:
sample_rate = 16000

duration = len(aaa)/sample_rate

print(f'Total duration: {duration:.2f}s')

for i in range(0, int(duration), 5):
  start = i*sample_rate
  end   = (i+5)*sample_rate
  print('duration from {:d} -- {:d}'.format(i, i+5))

  wav_data = aaa[start:end]
  reloaded_results = reloaded_model(wav_data)

  baby_sound = my_classes[tf.argmax(reloaded_results)]
  print(f'The main sound is: {baby_sound}')


Total duration: 255.28s
duration from 0 -- 5
The main sound is: laughing
duration from 5 -- 10
The main sound is: laughing
duration from 10 -- 15
The main sound is: crying_baby
duration from 15 -- 20
The main sound is: crying_baby
duration from 20 -- 25
The main sound is: laughing
duration from 25 -- 30
The main sound is: laughing
duration from 30 -- 35
The main sound is: crying_baby
duration from 35 -- 40
The main sound is: laughing
duration from 40 -- 45
The main sound is: laughing
duration from 45 -- 50
The main sound is: crying_baby
duration from 50 -- 55
The main sound is: crying_baby
duration from 55 -- 60
The main sound is: laughing
duration from 60 -- 65
The main sound is: laughing
duration from 65 -- 70
The main sound is: crying_baby
duration from 70 -- 75
The main sound is: laughing
duration from 75 -- 80
The main sound is: crying_baby
duration from 80 -- 85
The main sound is: laughing
duration from 85 -- 90
The main sound is: crying_baby
duration from 90 -- 95
The main sound

## Real-Time audio

In [150]:
import cv2
import wave
import pyaudio
import scipy
from moviepy.audio import AudioClip

p = pyaudio.PyAudio()

# sample format
FORMAT          = pyaudio.paInt16
# mono, change to 2 if you want stereo
CHANNELS        = 1
# 44100 samples per second
RATE            = 16000
# record period
RECORD_SECONDS  = 3
# set the chunk size of 1024 samples
CHUNK           = 4096 * 2

min_frames_to_process = int(RATE * 2.5)

chunks_required = int(np.ceil(min_frames_to_process // CHUNK))

# the file name output you want to record into
filename = "recorded.wav"

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                #output=True,
                frames_per_buffer=CHUNK)

while True:
    frames = []
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK, exception_on_overflow=False)
        
        #if you want to hear your voice while recording
        #stream.write(data)
        
        #result = np.fromstring(data, dtype=np.float16)
        #frames.append(result)
        
        frames.append(data)
    #npdata = np.hstack(frames)
    
    #npdata = np.array(npdata, dtype=np.float32)

    #wav_data = AudioClip.from_np(npdata, RATE)

    #check using model
    reloaded_results = reloaded_model(wave_arr)
    print(reloaded_results)
    baby_sound = my_classes[tf.argmax(reloaded_results)]
    print(f'The main sound is: {baby_sound}')
    
    #c = cv.WaitKey(7) % 0x100
    #if c == 27 or c == 10:
    #    break

# stop and close stream
stream.stop_stream()
stream.close()

# terminate pyaudio object
p.terminate()

wf = wave.open(filename, "wb")
# set the channels
wf.setnchannels(CHANNELS)
# set the sample format
wf.setsampwidth(p.get_sample_size(FORMAT))
# set the sample rate
wf.setframerate(RATE)
# write the frames as bytes
wf.writeframes(b"".join(frames))
# close the file
wf.close()

1
2
tf.Tensor([-1.5416976  4.2150908], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.3106263  3.677208 ], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.472271  4.151361], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.150538   2.5794582], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.6066098  4.1252203], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.4608643  3.9879856], shape=(2,), dtype=float32)
The main sound is: laughing
1
2
tf.Tensor([-1.4848602  4.0876384], shape=(2,), dtype=float32)
The main sound is: laughing


KeyboardInterrupt: 